# b) Ngonye Falls Synthetic Historic Flow Series

To produce a long time series (1924-2017) of daily flow for the Ngonye Falls site, a correlation is carried out between the overlapping portions of the *Zambezi River Authority* (ZRA) gauge at Ngonye Falls and portion of the record of the Victoria Falls gauge (2005-2016).

Once correlated, the full Victoria Falls gauge record is extrapolated to produce a synthetic record for Ngonye Falls.

## Procedure

From: **Mott MacDonald** - *Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D*

1. Calculate Flow Duration Curves (FDCs) from daily data for the following:
    1. The target site (Ngonye) for 2005/06 – 2016/17; 
    2. The analogue site (Victoria Falls) for the exact period of data at the target site (2005/06 – 2016/17);
    3. Analogue site for long-term record (1924/25 -2016/17); 
    
    
2. Compare the FDCs for the analogue site for the two periods and derive factors between the shorter and longer periods; 

3. Apply these factors to the target site FDC to produce an estimated long-term FDC for the target site;

4. For each daily flow value at the analogue site, determine its position on the long-term FDC (percentile); and,

5. Look up the flow value for this percentile from the estimated long-term FDC for the target site taking into account of the 11-day time lag.

*Water Year* is defined as running Oct to Sept as is usual in the region.

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| daily_gauge_vicfalls.csv        | Zambezi River Authority | Daily Flow for Victoria Falls 1924/25 - 2016/17              |
| daily_gauge_ngonye.csv | Zambezi River Authority   | Daily Stage (Level) for Ngonye Falls 2005/06 – 2016/17 |
|  Stage - Discharge equation for Ngonye Falls |  Mott MacDonald |Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D |


## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| ngonye_synthetic.csv  | Synthetic daily flow series for Ngonye  Falls 1924/25 - 2016/17  |
| ngonye_vicfalls_fdcs.csv           | Flow Duration Curves for Victoria Falls and Ngonye Falls |


## Parameters

In [10]:
input_data='./input_data/'
output_data='./output_data/'
#overlap_cutoff=datetime.datetime(2017, 9,30)
overlap_cutoff=datetime.datetime(2020, 5,31)

## Libraries

In [11]:
import numpy as np
import pandas as pd
import datetime

## Load Data
Load the Vic Falls data and add some helper columns.

In [12]:
vicfalls=pd.read_csv(input_data + "daily_gauge_vicfalls_2020.csv")
#vicfalls=pd.read_csv(input_data + "daily_gauge_vicfalls.csv")

vicfalls['Date']=pd.to_datetime(vicfalls['Date'],format="%d/%m/%Y")
vicfalls=vicfalls.set_index(pd.DatetimeIndex(vicfalls['Date']))
vicfalls=vicfalls.drop(['Date'],axis=1)

vicfalls=vicfalls.astype({'Flow': 'float64'})
vicfalls['Flow']=np.round(vicfalls['Flow'],4)

vicfalls.tail(5)

,Flow
Date,
2020-05-27,3453.1404
2020-05-28,3438.1311
2020-05-29,3393.3391
2020-05-30,3304.8137
2020-05-31,3217.6933


Load the Ngonye Falls level data and calculate flow based on the stage-discharge relationship:

\begin{equation*}
flow=1093.0355*(level-2.85)^{1.659}
\end{equation*}

Add some helper columns.

In [13]:
ngo=pd.read_csv(input_data+ "daily_gauge_ngonye_2020.csv")
#ngo=pd.read_csv(input_data+ "daily_gauge_ngonye.csv")
ngo['Date']=pd.to_datetime(ngo['Date'],format="%d/%m/%Y")
ngo=ngo.set_index(pd.DatetimeIndex(ngo['Date']))
ngo=ngo.drop(['Date'],axis=1)
ngo['Flow']=np.round(1093.0355*(ngo['Level']-2.85)**1.659,4)
ngo['VicFalls']=vicfalls['Flow']
ngo.tail(5)

,Level,Flow,VicFalls
Date,,,
2020-05-27,4.514,2544.0616,3453.1404
2020-05-28,4.488,2478.4549,3438.1311
2020-05-29,4.464,2418.5007,3393.3391
2020-05-30,4.442,2364.0563,3304.8137
2020-05-31,4.422,2314.9895,3217.6933


## Flow Duration Curve

Build the Flow Duration Curve table in 0.1% exceedance increments.

Add FDC flows for the full Vic Falls timeseries.

In [14]:
fdc=pd.DataFrame({'Exceedance': np.arange(0,1.001,0.001)}).set_index('Exceedance')
fdc['VicFalls_full']=np.round(np.quantile(vicfalls.loc[vicfalls.index<=overlap_cutoff]['Flow'],((1-fdc.index))),4)
fdc

,VicFalls_full
Exceedance,
0.000,9435.5576
0.001,8445.5052
0.002,7520.3744
0.003,6685.2356
0.004,6234.1343
...,...
0.996,123.7663
0.997,113.3633
0.998,106.8565


Add flows for Ngonye to the FDC.

Add flows for the portion of the Vic Falls series that overlaps with the Ngonye Series.

In [15]:
fdc['Ngonye_gauged']=np.round(np.percentile(ngo.loc[ngo.index<=overlap_cutoff]['Flow'],((1-fdc.index)*100)),4)
fdc['VicFalls_overlap']=np.round(np.percentile(ngo.loc[ngo.index<=overlap_cutoff]['VicFalls'],((1-fdc.index)*100)),4)

## Vic Falls Ratio
Calculate the ratio of Vic Falls flows for the whole series and the overallping protion across the FDC.

Smooth that ratio (0.7% moving average) except at the tails of the FDC.

In [16]:
fdc['VicFalls_factor']=fdc['VicFalls_full']/fdc['VicFalls_overlap']
fdc['VicFalls_factor_smooth']=fdc['VicFalls_factor'].rolling(7,center=True).mean()
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name<0.01 else x['VicFalls_factor_smooth'])),axis=1)
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name>0.99 else x['VicFalls_factor_smooth'])),axis=1)

## Scaled FDC
Produce a scaled FDC for Ngonye by using the factors calculated for the Vic Falls data.

In [17]:
fdc['Ngonye_scaled']=fdc['Ngonye_gauged']*fdc['VicFalls_factor_smooth']

## Conversion
Calculate a conversion factor for each row of the FDC between the scaled Ngonye FDC flows and the full Vic Falls series FDC.

In [18]:
fdc['Conversion']=fdc['Ngonye_scaled']/fdc['VicFalls_full']
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion
Exceedance,,,,,,,
0.000,9435.5576,5935.9208,5650.5399,1.669851,1.669851,9912.101075,1.050505
0.001,8445.5052,5809.1639,5582.9446,1.512733,1.512733,8787.714627,1.040520
0.002,7520.3744,5637.7169,5436.4811,1.383317,1.383317,7798.747217,1.037016
0.003,6685.2356,5493.5563,5292.3528,1.263188,1.263188,6939.393411,1.038018
0.004,6234.1343,5312.6645,5150.5566,1.210381,1.210381,6430.346573,1.031474
...,...,...,...,...,...,...,...
0.996,123.7663,154.4514,140.1103,0.883349,0.883349,136.434497,1.102356
0.997,113.3633,153.2661,129.4946,0.875429,0.875429,134.173555,1.183571
0.998,106.8565,152.4361,117.1741,0.911946,0.911946,139.013554,1.300937


Lookup the conversion factors from the FDC based on flow and give each record in the full Vic Falls series its corresponding conversion factor. 

In [19]:
if 'Conversion' in vicfalls.columns:
    display(vicfalls.columns)
    vicfalls=vicfalls.drop(['Conversion','Exceedance'],axis=1)

#vicfalls['tmp_Flow']=100000-vicfalls['Flow']
#fdc['tmp_Flow']=100000-fdc['VicFalls_full']
fdc['ExceedanceInv']=1-fdc.index

tmp=pd.merge_asof(vicfalls.reset_index().sort_values('Flow',ascending=True),fdc.reset_index().sort_values(['VicFalls_full','ExceedanceInv'],ascending=True),left_on='Flow',right_on='VicFalls_full').set_index('Date')
vicfalls['Conversion']=tmp['Conversion']
vicfalls['Exceedance']=tmp['Exceedance']



## Ngonye Sythetic
Prepare the full  synthetic series for Ngonye by applying the 11 day lag to the Vic Falls series and the conversion factors calculated previoulsy.

In [20]:

ngonye_synth=pd.DataFrame(index=vicfalls.index)

ngonye_synth['LaggedDate']=ngonye_synth.index+pd.DateOffset(days=11)
ngonye_synth['VicFalls']=ngonye_synth.join(vicfalls,on='LaggedDate')['Flow']
ngonye_synth['Conversion']=ngonye_synth.join(vicfalls,on='LaggedDate')['Conversion']
ngonye_synth['Flow']=ngonye_synth['Conversion'] * ngonye_synth['VicFalls']

vicfalls.loc[vicfalls.index==datetime.datetime(1924,12,16)]

,Flow,Conversion,Exceedance
Date,,,
1924-12-16,231.0479,0.973372,0.914


Remove extra columns and delete from the bottom to align to the water year.

In [21]:
ngonye_synth=ngonye_synth.dropna()
#ngonye_synth=ngonye_synth.drop(ngonye_synth.loc['2017-10-01':].index)
ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow
Date,,,,
1924-10-01,1924-10-12,100.0000,1.407305,140.730461
1924-10-02,1924-10-13,100.0000,1.407305,140.730461
1924-10-03,1924-10-14,100.0000,1.407305,140.730461
1924-10-04,1924-10-15,100.0000,1.407305,140.730461
1924-10-05,1924-10-16,100.0000,1.407305,140.730461
...,...,...,...,...
2020-05-16,2020-05-27,3453.1404,1.037022,3580.983662
2020-05-17,2020-05-28,3438.1311,1.035186,3559.103900
2020-05-18,2020-05-29,3393.3391,1.028636,3490.510716


Add the new synthetic Ngonye flow series to the FDC.

In [22]:
fdc['Ngonye_synthetic']=np.percentile(ngonye_synth['Flow'],((1-fdc.index)*100))

tmp=pd.merge_asof(ngonye_synth.sort_values('Flow').reset_index(),fdc.sort_values('Ngonye_synthetic').reset_index(),left_on='Flow',right_on='Ngonye_synthetic').set_index('Date')
ngonye_synth['Exceedance']=tmp['Exceedance']

# Prepare Export
Prepare a subset of the FDC for export

In [23]:
fdc_out=fdc.loc[:,['VicFalls_full','Ngonye_gauged','Ngonye_synthetic','Conversion']]
fdc_out

,VicFalls_full,Ngonye_gauged,Ngonye_synthetic,Conversion
Exceedance,,,,
0.000,9435.5576,5935.9208,9912.101075,1.050505
0.001,8445.5052,5809.1639,8760.450163,1.040520
0.002,7520.3744,5637.7169,7798.747217,1.037016
0.003,6685.2356,5493.5563,6905.534712,1.038018
0.004,6234.1343,5312.6645,6402.292338,1.031474
...,...,...,...,...
0.996,123.7663,154.4514,145.266063,1.102356
0.997,113.3633,153.2661,144.428820,1.183571
0.998,106.8565,152.4361,141.399969,1.300937


## Export

In [24]:
#ngonye_synth.to_csv(output_data + 'ngonye_synthetic.csv')
#fdc_out.to_csv(output_data + 'ngonye_vicfalls_fdcs.csv')
ngonye_synth.to_csv(output_data + 'ngonye_synthetic_2020.csv')
fdc_out.to_csv(output_data + 'ngonye_vicfalls_fdcs_2020.csv')

In [25]:
ngonye_synth.loc[ngonye_synth.index=='1925-02-05']

,LaggedDate,VicFalls,Conversion,Flow,Exceedance
Date,,,,,
1925-02-05,1925-02-16,1238.2939,0.933909,1156.453404,0.289


In [26]:
fdc.loc[(fdc.index<=0.291) & (fdc.index>=0.288)].reset_index().sort_values(['VicFalls_full','ExceedanceInv'],ascending=[True,True])

,Exceedance,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion,ExceedanceInv,Ngonye_synthetic
3,0.291,1229.0427,1479.9209,1586.6955,0.774593,0.774751,1146.569578,0.932896,0.709,1146.569578
2,0.290,1234.8221,1487.2830,1595.7550,0.773817,0.773920,1151.037409,0.932148,0.710,1151.037409
1,0.289,1238.2939,1493.6742,1598.6159,0.774604,0.772697,1154.157669,0.932055,0.711,1156.453404
0,0.288,1238.2939,1497.5437,1610.5937,0.768843,0.772233,1156.453404,0.933909,0.712,1156.540221
